In [7]:
import os
import numpy as np
import segyio
from scipy import stats
from multiprocessing import Pool
np.set_printoptions(threshold=np.inf) # Print the entire dataframe

# Data to use

In [8]:
# Seismic data: SEG-Y format
near_stack = os.path.join("..", "data", "NS2950-2180_3000-2230.sgy")
mid_stack = os.path.join("..", "data", "MS2950-2180_3000-2230.sgy")
far_stack = os.path.join("..", "data", "FS2950-2180_3000-2230.sgy")

# To store G and I
g_i = os.path.join("..", "data", "GradInter.sgy")
# Wells data: txt format
wells_path = '../data/wells_info.txt'

# Store the seismic data in a list
gather_path = [near_stack, mid_stack, far_stack]

# Input parameters to work with


In [9]:
# Name of the survery to display in the results
seismic_survey = 'POSEIDON 3D'

# Name and path of the merged SEG-Y file
merged_stacks = os.path.join("..", "data", "NMF2950-2180_3000-2230.sgy")

# Gradient file (SEGY)
gradient_segy = os.path.join("..", "data", "gradient.sgy")
# Intercept file (SEGY)
intercept_segy = os.path.join("..", "data", "intercept.sgy")

# Angle between the gathers given
angle_bet_gathers = 12

# Making a list with the angle of the gathers
angle_list = []
angle = angle_bet_gathers
for file in range(len(gather_path)):
    angle_list = angle_list + [angle]
    angle = angle + angle_bet_gathers

# Merging the partial angle stacks


In [10]:
def merging_stacks(list_of_gathers, merged_file_path, list_of_angles):
    
    """
    
    source1: https://github.com/equinor/segyio/blob/master/python/examples/make-file.py
    source2: https://github.com/equinor/segyio/blob/master/python/segyio/tracefield.py
    
    """
    
    # Validating the files
#    for file in list_of_gathers:
#        if file_validation(file) == True:
#            print(f"{file} has been validated")
#        else:
#            return(f"Unsuccessful validation: {file_validation(file)}")
    
    # Making the offset array
    offsts = np.array(list_of_angles)

    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f, segyio.open(list_of_gathers[1]) as g, segyio.open(list_of_gathers[2]) as h:
        
        # Spec function to build the new segy
        spec = segyio.spec()
        spec.sorting = f.sorting
        spec.format = 1
        spec.samples = f.samples
        spec.ilines = f.ilines
        spec.xlines = f.xlines
        spec.offsets = offsts

        # Initializing the merged one
        with segyio.create(merged_file_path, spec) as s:
        
            # Index for the merge headers and trace
            merge_index = 0
            # Index for the original file's headers and trace
            stack_index = 0
            # For loop to set parameters according to the seismic lines and offset
            for il in spec.ilines:
                for xl in spec.xlines:
                    for offset in spec.offsets:
                        # Assigning headers [byte]
                        s.header[merge_index] = {segyio.su.tracl  : f.header[stack_index][1],
                                                 segyio.su.tracr  : f.header[stack_index][5],
                                                 segyio.su.fldr   : f.header[stack_index][9],
                                                 segyio.su.cdp    : f.header[stack_index][21],
                                                 segyio.su.cdpt   : f.header[stack_index][25],
                                                 segyio.su.offset : offset, # 37
                                                 segyio.su.scalco : f.header[stack_index][71],
                                                 segyio.su.ns     : f.header[stack_index][115],
                                                 segyio.su.dt     : f.header[stack_index][117],
                                                 segyio.su.cdpx   : f.header[stack_index][181],
                                                 segyio.su.cdpy   : f.header[stack_index][185],
                                                 segyio.su.iline  : il,  # 189
                                                 segyio.su.xline  : xl}  # 193
                        if offset == list_of_angles[0]:
                            stack = f # near
                        if offset == list_of_angles[1]:
                            stack = g # mid
                        if offset == list_of_angles[2]:
                            stack = h # far                   
                        
                        # Copying the amplitudes from each stack
                        s.trace[merge_index] = stack.trace[stack_index]
                        
                        merge_index += 1
                    stack_index += 1
                
    return (f"Successful merge. New SEG-Y file path: {merged_file_path}")

In [11]:
merging_stacks(gather_path, merged_stacks, angle_list)

'Successful merge. New SEG-Y file path: ../data/NMF2950-2180_3000-2230.sgy'

# First version of multiprocess avo attributes

## Making the SEG-Y files to store the AVO attributes [26 min]

In [12]:
def avo_storing_files(list_of_gathers, intercept_path, gradient_path): 
    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f:
        
        # Spec function to build the new segy
        spec = segyio.tools.metadata(f)

        # Initializing the merged one
        with segyio.create(gradient_path, spec) as gradient, segyio.create(intercept_path, spec) as intercept:

            # Assigning headers [byte]
            gradient.text[0], intercept.text[0] = f.text[0], f.text[0]
            gradient.bin, intercept.bin = f.bin, f.bin
            gradient.header, intercept.header = f.header, f.header
            gradient.trace, intercept.trace = f.trace, f.trace
            
    return (f"Successful construction. (Gradient, Intercept) path: ({intercept_path}, {gradient_path})")

In [13]:
avo_storing_files(gather_path, intercept_segy, gradient_segy)

'Successful construction. (Gradient, Intercept) path: (../data/intercept.sgy, ../data/gradient.sgy)'

## Function to give arguments to the pool

In [14]:
def input_generator(gather_file, list_of_angles, intercept_file, gradient_file):
    """
    
    """
    index = 0
    with segyio.open(gather_file, "r") as gather:
        for iline_number in gather.ilines:
            for xline_number in gather.xlines:
                    yield [gather_file, list_of_angles, iline_number, xline_number, index, intercept_file, gradient_file]
                    index += 1
                    
# test it
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(row)

In [16]:
input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)

for row in input_args:
    print(row)

['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2180, 0, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2181, 1, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2182, 2, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2183, 3, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2184, 4, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2185, 5, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2186, 6, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2187, 7, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2950, 2188,

['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2180, 765, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2181, 766, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2182, 767, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2183, 768, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2184, 769, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2185, 770, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2186, 771, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2965, 2187, 772, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 

['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2207, 1302, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2208, 1303, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2209, 1304, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2210, 1305, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2211, 1306, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2212, 1307, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2213, 1308, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2975, 2214, 1309, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [

['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2214, 2023, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2215, 2024, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2216, 2025, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2217, 2026, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2218, 2027, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2219, 2028, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2220, 2029, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [12, 24, 36], 2989, 2221, 2030, '../data/intercept.sgy', '../data/gradient.sgy']
['../data/NMF2950-2180_3000-2230.sgy', [

## Function to compute the AVO attributes by trace

In [17]:
from scipy import stats
def  AVO_attributes_computation(gather_file, iline_number, xline_number, list_of_angles):
    """
    
    """
    sins = []
    for angle in angle_list:
        sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]
    
    with segyio.open(gather_file, "r") as gather:
        for time_slice in range(len(gather.samples)):
            amp = []
            for angle in list_of_angles:
                amp += [gather.gather[iline_number, xline_number, angle][time_slice]]
            
            # Regression
            gradient, intercept, r_value, p_value, std_error = stats.linregress(sins, amp)
            
            yield[intercept, gradient]

# Test it
#avo_attributes = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)
#for row in avo_attributes:
#    print(row)

### Fnction vectorizada

In [20]:
avo_attributes = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)

for row in avo_attributes:

    print(row)

[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]

[-46.062655030302444, 103.1987853601835]
[-19.13425960739475, 16.967824470015177]
[15.379166763483397, -56.18930843261866]
[4.738299249220761, 14.048640203895566]
[8.893956292988548, 9.596857828785184]
[7.667463073118212, -1.8088648638238198]
[9.723893920210488, -16.550798146283803]
[30.09325967320322, -115.99634659288496]
[-54.084646747475574, 22.112899923689973]
[-132.61589277377195, 214.46708282312505]
[12.601056141671286, -32.12678407620653]
[136.2907709758863, -167.5931130814949]
[77.87494998331877, -8.345778857591066]
[-17.42579636166643, 114.18748172906115]
[-10.86574039260525, -16.967824470015177]
[-27.106044660685768, 9.596857828785184]
[-38.95439622412602, 86.37172045906665]
[26.52101371271532, -22.67068581631968]
[58.14287348679672, -110.85127113921016]
[-18.86113750459712, -29.6246261338183]
[-99.46620225347645, 83.7288029477801]
[-66.3319426460817, 66.76097847776491]
[51.02523240022857, -70.51421539134726]
[102.41658939355727, -88.87387840145489]
[45.88474765594934, 34.910

[12.506608145286467, -53.27012416649905]
[6.631660665362769, -44.92434530890798]
[-10.101440819003322, -3.0599438350179367]
[-23.39595988133683, 31.016464673910747]
[-19.676670279645947, 27.12246819143006]
[0.235700187979492, -13.90788063499724]
[15.810929410203537, -44.09029266144523]
[8.778110621812111, -24.062524356412123]
[-12.627056942889613, 32.26754364510486]
[-18.685877009336522, 52.436071519036304]
[-3.1320448658576483, 9.737617397683511]
[14.886369427988534, -40.88958925752897]
[11.447788317258134, -33.101596292567606]
[-5.926775081379974, 10.430910476247929]
[-12.752109343756626, 23.92176478751379]
[-2.0236120587008495, -5.28583502257313]
[4.773507018548244, -11.405722692609004]
[-4.447788198048845, 33.101596292567606]
[-13.539427172096474, 73.29789247153217]
[-6.839145072168254, 51.461259302675224]
[3.537806830568752, 2.5021579423882354]
[1.9527758825983015, -10.57167004514626]
[-13.318132074708675, 28.790573486355555]
[-21.197683038686947, 49.793154007749735]
[-6.695678496

[0.006818225591678662, -19.887008736134785]
[3.527831223051259, -42.55769455245446]
[3.6102633481990196, -57.44038740381277]
[1.3555547047825502, -61.4751434551918]
[-0.5666165349149797, -63.701034642746976]
[-5.238682972878481, -49.23536811512004]
[-12.4317631235573, -12.099015771173418]
[-16.855938905277426, 26.288415543967314]
[-13.525616600723447, 35.32748748012279]
[-5.393571794470446, 25.58987008243929]
[3.3845387677370216, 1.52734572602716]
[9.547014037096488, -22.811445385218008]
[12.452391682103423, -45.75839795637073]
[8.527831401865193, -42.55769455245446]
[2.6246684387907147, -26.840949053633405]
[-0.5614188892696013, -7.787992964961364]
[-0.08004326342099399, 9.456098259886854]
[-2.192905553651994, 38.93996482480682]
[-7.152499900260551, 69.39864360608786]
[-12.779556843150164, 101.66618725119272]
[-16.170913836478867, 120.02585026130035]
[-13.675901401819033, 97.495924013879]
[-4.862757130869104, 46.1754242801021]
[4.84894655949608, -8.205019288692737]
[12.044240855665551

[-7.131870864877271, 11.541229878543717]
[-5.900774280161647, 10.290150907349599]
[-3.7711182760307658, 5.979128101137545]
[0.155656864953853, -4.451782375110384]
[4.082432005938472, -14.882692851358316]
[8.009206968109154, -25.313603327606238]
[9.24030379124336, -26.564682298800363]
[8.67428106029131, -21.695873599958603]
[6.311139967345908, -10.707177231080973]
[4.436192368212921, -2.3613983734899056]
[3.537632948797664, 0.6985454615280313]
[3.6154612322629784, -1.52734572602716]
[4.669677695446023, -9.039071936155482]
[5.3135284714448066, -16.13377182255243]
[3.261700929593174, -14.048640203895566]
[-0.17688024074186925, -6.260647238934204]
[-4.591849173562129, 6.813180748600289]
[-8.596452776825997, 19.46998241240341]
[-10.393571615656512, 25.58987008243929]
[-9.572840718125146, 24.755817434976535]
[-8.341744133409524, 23.50473846378242]
[-5.3135284714448066, 16.13377182255243]
[-2.2853129882940237, 8.76280518132244]
[0.3325371056957224, 1.8088648638238192]
[3.4385811107303987, -7.

## Function to store the AVO attributes by trace

In [36]:
def store_AVO_attributes(generator_list):
    """
    
    """
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    
    to_store = AVO_attributes_computation(gather_file, iline_number, xline_number, 
                                          list_of_angles)
    cont = 0
    with segyio.open(intercept_file, "r+") as intercept, segyio.open(gradient_file, "r+") as gradient:
        intercept_array = np.zeros([len(intercept.samples), ], dtype = "float32")
        gradient_array = np.zeros([len(gradient.samples), ], dtype = "float32")
        
        # Inspecting the generator
        for attribute in to_store:
            intercept_array[cont] = attribute[0]
            gradient_array[cont] = attribute[1]
            cont += 1
        
        # Storing
        intercept.trace[index] = intercept_array
        gradient.trace[index] = gradient_array
        
        print(f"AVO attributes for trace [{iline_number},{xline_number}, {index}] has been stored successfully")
        return()
    
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(store_AVO_attributes(row))

## Integrating the previous functions [MP]

In [37]:
def avo_storeputation(gather_file, list_of_angles, intercept_file, gradient_file):
    
    input_data = input_generator(gather_file, list_of_angles, intercept_file, gradient_file)
    
    if __name__ == "__main__":
        p = Pool()
        p.map(store_AVO_attributes, input_data)

In [38]:
avo_storeputation(merged_stacks, angle_list, intercept_segy, gradient_segy)

AVO attributes for trace [2956,2200, 326] has been stored successfully
AVO attributes for trace [2950,2180, 0] has been stored successfully
AVO attributes for trace [2953,2190, 163] has been stored successfully
AVO attributes for trace [2959,2210, 489] has been stored successfully
AVO attributes for trace [2950,2181, 1] has been stored successfully
AVO attributes for trace [2956,2201, 327] has been stored successfully
AVO attributes for trace [2953,2191, 164] has been stored successfully
AVO attributes for trace [2959,2211, 490] has been stored successfully
AVO attributes for trace [2950,2182, 2] has been stored successfully
AVO attributes for trace [2953,2192, 165] has been stored successfully
AVO attributes for trace [2956,2202, 328] has been stored successfully


Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Process ForkPoolWorker-18:
Traceback (most recent call last):
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/lsd/anaconda3/envs/tesis/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/lsd/anaconda3/envs/te

# Second version of multiprocess avo attributes
What if we modify AVO_attributes_computation to yield a numpy array instead of a list?

## Function to give arguments to the pool

In [176]:
def input_generator(gather_file, list_of_angles, intercept_file, gradient_file):
    """
    
    """
    index = 0
    with segyio.open(gather_file, "r") as gather:
        for iline_number in gather.ilines:
            for xline_number in gather.xlines:
                    yield [gather_file, list_of_angles, iline_number, xline_number, index, intercept_file, gradient_file]
                    index += 1
# test it
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(row)

In [177]:
input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)

<generator object input_generator at 0xa8d2102c>

## Function to compute the AVO attributes by trace (MODIFIED)

In [178]:
from scipy import stats
def  AVO_attributes_computation(gather_file, iline_number, xline_number, list_of_angles):
 
    sins = []
    for angle in angle_list:
        sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]

    with segyio.open(gather_file, "r") as gather:
        
        g = np.zeros([len(gather.samples),], dtype="float32")
        i = np.zeros([len(gather.samples),], dtype="float32")
        for time_slice in range(len(gather.samples)):
            amp = []
            for angle in list_of_angles:
                amp += [gather.gather[iline_number, xline_number, angle][time_slice]]

            # Regression
            gradient, intercept, r_value, p_value, std_error = stats.linregress(sins, amp)

            g[time_slice] = gradient
            i[time_slice] = intercept
        
    yield[i, g]
# Test it                          [2950 - 3000 // 2180 - 2230]
#avo_attributes = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)
#for row in avo_attributes:
#    print(row)

In [175]:
xxx = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)
for array in xxx:
    print(len(array))


2


## Function to store the AVO attributes by trace

In [180]:
def store_AVO_attributes(generator_list):
    """
    
    """
    
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    
    to_store = AVO_attributes_computation(gather_file, iline_number, xline_number, 
                                          list_of_angles)
    
    with segyio.open(intercept_file, "r+") as intercept, segyio.open(gradient_file, "r+") as gradient:
        
        for array in to_store:
            # Storing
            intercept.trace[index] = array[0]
            gradient.trace[index] = array[1]
        
        print(f"AVO attributes for trace [{iline_number},{xline_number}] has been stored successfully")
        return()
    
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(store_AVO_attributes(row))

In [181]:
def avo_storeputation(gather_file, list_of_angles, intercept_file, gradient_file):
    
    input_data = input_generator(gather_file, list_of_angles, intercept_file, gradient_file)
    
    if __name__ == "__main__":
        p = Pool()
        p.map(store_AVO_attributes, input_data)

In [182]:
avo_storeputation(merged_stacks, angle_list, intercept_segy, gradient_segy)

AVO attributes for trace [2953,2190] has been stored successfully
AVO attributes for trace [2950,2180] has been stored successfully
AVO attributes for trace [2959,2210] has been stored successfully
AVO attributes for trace [2956,2200] has been stored successfully
AVO attributes for trace [2950,2181] has been stored successfully
AVO attributes for trace [2959,2211] has been stored successfully
AVO attributes for trace [2953,2191] has been stored successfully
AVO attributes for trace [2956,2201] has been stored successfully
AVO attributes for trace [2950,2182] has been stored successfully
AVO attributes for trace [2959,2212] has been stored successfully
AVO attributes for trace [2953,2192] has been stored successfully
AVO attributes for trace [2956,2202] has been stored successfully
AVO attributes for trace [2950,2183] has been stored successfully
AVO attributes for trace [2959,2213] has been stored successfully
AVO attributes for trace [2953,2193] has been stored successfully
AVO attrib

AVO attributes for trace [2957,2180] has been stored successfully
AVO attributes for trace [2953,2221] has been stored successfully
AVO attributes for trace [2950,2212] has been stored successfully
AVO attributes for trace [2957,2181] has been stored successfully
AVO attributes for trace [2960,2190] has been stored successfully
AVO attributes for trace [2953,2222] has been stored successfully
AVO attributes for trace [2950,2213] has been stored successfully
AVO attributes for trace [2957,2182] has been stored successfully
AVO attributes for trace [2960,2191] has been stored successfully
AVO attributes for trace [2953,2223] has been stored successfully
AVO attributes for trace [2950,2214] has been stored successfully
AVO attributes for trace [2957,2183] has been stored successfully
AVO attributes for trace [2960,2192] has been stored successfully
AVO attributes for trace [2953,2224] has been stored successfully
AVO attributes for trace [2950,2215] has been stored successfully
AVO attrib

AVO attributes for trace [2951,2192] has been stored successfully
AVO attributes for trace [2954,2201] has been stored successfully
AVO attributes for trace [2960,2221] has been stored successfully
AVO attributes for trace [2957,2212] has been stored successfully
AVO attributes for trace [2951,2193] has been stored successfully
AVO attributes for trace [2954,2202] has been stored successfully
AVO attributes for trace [2960,2222] has been stored successfully
AVO attributes for trace [2957,2213] has been stored successfully
AVO attributes for trace [2951,2194] has been stored successfully
AVO attributes for trace [2954,2203] has been stored successfully
AVO attributes for trace [2960,2223] has been stored successfully
AVO attributes for trace [2957,2214] has been stored successfully
AVO attributes for trace [2954,2204] has been stored successfully
AVO attributes for trace [2951,2195] has been stored successfully
AVO attributes for trace [2960,2224] has been stored successfully
AVO attrib

AVO attributes for trace [2951,2223] has been stored successfully
AVO attributes for trace [2961,2201] has been stored successfully
AVO attributes for trace [2958,2192] has been stored successfully
AVO attributes for trace [2955,2182] has been stored successfully
AVO attributes for trace [2951,2224] has been stored successfully
AVO attributes for trace [2961,2202] has been stored successfully
AVO attributes for trace [2958,2193] has been stored successfully
AVO attributes for trace [2955,2183] has been stored successfully
AVO attributes for trace [2951,2225] has been stored successfully
AVO attributes for trace [2961,2203] has been stored successfully
AVO attributes for trace [2958,2194] has been stored successfully
AVO attributes for trace [2955,2184] has been stored successfully
AVO attributes for trace [2951,2226] has been stored successfully
AVO attributes for trace [2958,2195] has been stored successfully
AVO attributes for trace [2961,2204] has been stored successfully
AVO attrib

AVO attributes for trace [2958,2223] has been stored successfully
AVO attributes for trace [2955,2213] has been stored successfully
AVO attributes for trace [2962,2181] has been stored successfully
AVO attributes for trace [2952,2204] has been stored successfully
AVO attributes for trace [2958,2224] has been stored successfully
AVO attributes for trace [2955,2214] has been stored successfully
AVO attributes for trace [2962,2182] has been stored successfully
AVO attributes for trace [2952,2205] has been stored successfully
AVO attributes for trace [2958,2225] has been stored successfully
AVO attributes for trace [2955,2215] has been stored successfully
AVO attributes for trace [2962,2183] has been stored successfully
AVO attributes for trace [2952,2206] has been stored successfully
AVO attributes for trace [2958,2226] has been stored successfully
AVO attributes for trace [2955,2216] has been stored successfully
AVO attributes for trace [2962,2184] has been stored successfully
AVO attrib

...

AVO attributes for trace [2988,2211] has been stored successfully
AVO attributes for trace [2998,2186] has been stored successfully
AVO attributes for trace [2991,2220] has been stored successfully
AVO attributes for trace [2994,2228] has been stored successfully
AVO attributes for trace [2998,2187] has been stored successfully
AVO attributes for trace [2988,2212] has been stored successfully
AVO attributes for trace [2991,2221] has been stored successfully
AVO attributes for trace [2994,2229] has been stored successfully
AVO attributes for trace [2998,2188] has been stored successfully
AVO attributes for trace [2988,2213] has been stored successfully
AVO attributes for trace [2991,2222] has been stored successfully
AVO attributes for trace [2994,2230] has been stored successfully
AVO attributes for trace [2998,2189] has been stored successfully
AVO attributes for trace [2988,2214] has been stored successfully
AVO attributes for trace [2991,2223] has been stored successfully
AVO attrib

AVO attributes for trace [2995,2207] has been stored successfully
AVO attributes for trace [2992,2200] has been stored successfully
AVO attributes for trace [2989,2192] has been stored successfully
AVO attributes for trace [2998,2218] has been stored successfully
AVO attributes for trace [2995,2208] has been stored successfully
AVO attributes for trace [2992,2201] has been stored successfully
AVO attributes for trace [2989,2193] has been stored successfully
AVO attributes for trace [2998,2219] has been stored successfully
AVO attributes for trace [2995,2209] has been stored successfully
AVO attributes for trace [2992,2202] has been stored successfully
AVO attributes for trace [2989,2194] has been stored successfully
AVO attributes for trace [2998,2220] has been stored successfully
AVO attributes for trace [2995,2210] has been stored successfully
AVO attributes for trace [2992,2203] has been stored successfully
AVO attributes for trace [2989,2195] has been stored successfully
AVO attrib

AVO attributes for trace [2989,2223] has been stored successfully
AVO attributes for trace [2996,2187] has been stored successfully
AVO attributes for trace [2999,2198] has been stored successfully
AVO attributes for trace [2993,2181] has been stored successfully
AVO attributes for trace [2996,2188] has been stored successfully
AVO attributes for trace [2989,2224] has been stored successfully
AVO attributes for trace [2999,2199] has been stored successfully
AVO attributes for trace [2993,2182] has been stored successfully
AVO attributes for trace [2989,2225] has been stored successfully
AVO attributes for trace [2996,2189] has been stored successfully
AVO attributes for trace [2999,2200] has been stored successfully
AVO attributes for trace [2993,2183] has been stored successfully
AVO attributes for trace [2996,2190] has been stored successfully
AVO attributes for trace [2989,2226] has been stored successfully
AVO attributes for trace [2999,2201] has been stored successfully
AVO attrib

AVO attributes for trace [2996,2218] has been stored successfully
AVO attributes for trace [2993,2212] has been stored successfully
AVO attributes for trace [2999,2229] has been stored successfully
AVO attributes for trace [2990,2204] has been stored successfully
AVO attributes for trace [2996,2219] has been stored successfully
AVO attributes for trace [2993,2213] has been stored successfully
AVO attributes for trace [2999,2230] has been stored successfully
AVO attributes for trace [2990,2205] has been stored successfully
AVO attributes for trace [2996,2220] has been stored successfully
AVO attributes for trace [2993,2214] has been stored successfully
AVO attributes for trace [3000,2180] has been stored successfully
AVO attributes for trace [2990,2206] has been stored successfully
AVO attributes for trace [2996,2221] has been stored successfully
AVO attributes for trace [2993,2215] has been stored successfully
AVO attributes for trace [3000,2181] has been stored successfully
AVO attrib

AVO attributes for trace [2997,2198] has been stored successfully
AVO attributes for trace [2994,2193] has been stored successfully
AVO attributes for trace [2991,2184] has been stored successfully
AVO attributes for trace [3000,2209] has been stored successfully
AVO attributes for trace [2997,2199] has been stored successfully
AVO attributes for trace [2994,2194] has been stored successfully
AVO attributes for trace [2991,2185] has been stored successfully
AVO attributes for trace [3000,2210] has been stored successfully
AVO attributes for trace [2997,2200] has been stored successfully
AVO attributes for trace [2994,2195] has been stored successfully
AVO attributes for trace [2991,2186] has been stored successfully
AVO attributes for trace [3000,2211] has been stored successfully
AVO attributes for trace [2997,2201] has been stored successfully
AVO attributes for trace [2994,2196] has been stored successfully
AVO attributes for trace [2991,2187] has been stored successfully
AVO attrib

# What if we merge computing and storing functions? [30 min +]
this way, we don't have the need to store the data in arrays to pass it to another function

In [7]:
from scipy import stats
def  AVO_attributes_computation(generator_list):
    
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    
    sins = []
    for angle in list_of_angles:
        sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]

    with segyio.open(gather_file, "r") as gather:
        with segyio.open(intercept_file, "r+") as i, segyio.open(gradient_file, "r+") as g:
            gradient_array = np.zeros([len(gather.samples),], dtype="float32")
            intercept_array = np.zeros([len(gather.samples),], dtype="float32")
            for time_slice in range(len(gather.samples)):
                amp = []
                for angle in list_of_angles:
                    amp += [gather.gather[iline_number, xline_number, angle][time_slice]]

                # Regression
                gradient, intercept, r_value, p_value, std_error = stats.linregress(sins, amp)

                gradient_array[time_slice] = gradient
                intercept_array[time_slice] = intercept
                
                # Storing
                i.trace[index] = intercept_array
                g.trace[index] = gradient_array
            
    print(f"AVO attributes for trace [{iline_number},{xline_number}] has been stored successfully")
    return()
# Test it
#avo_attributes = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)
#for row in avo_attributes:
#    print(row)

In [8]:
def avo_storeputation(gather_file, list_of_angles, intercept_file, gradient_file):
    
    input_data = input_generator(gather_file, list_of_angles, intercept_file, gradient_file)
    
    if __name__ == "__main__":
        p = Pool(4)
        p.map(AVO_attributes_computation, input_data)

In [9]:
avo_storeputation(merged_stacks, angle_list, intercept_segy, gradient_segy)

AVO attributes for trace [2953,2190] has been stored successfully
AVO attributes for trace [2956,2200] has been stored successfully
AVO attributes for trace [2950,2180] has been stored successfully
AVO attributes for trace [2959,2210] has been stored successfully
AVO attributes for trace [2953,2191] has been stored successfully
AVO attributes for trace [2956,2201] has been stored successfully
AVO attributes for trace [2950,2181] has been stored successfully
AVO attributes for trace [2959,2211] has been stored successfully
AVO attributes for trace [2953,2192] has been stored successfully
AVO attributes for trace [2956,2202] has been stored successfully
AVO attributes for trace [2959,2212] has been stored successfully
AVO attributes for trace [2950,2182] has been stored successfully
AVO attributes for trace [2953,2193] has been stored successfully
AVO attributes for trace [2950,2183] has been stored successfully
AVO attributes for trace [2959,2213] has been stored successfully
AVO attrib

AVO attributes for trace [2953,2221] has been stored successfully
AVO attributes for trace [2957,2180] has been stored successfully
AVO attributes for trace [2950,2211] has been stored successfully
AVO attributes for trace [2960,2191] has been stored successfully
AVO attributes for trace [2953,2222] has been stored successfully
AVO attributes for trace [2957,2181] has been stored successfully
AVO attributes for trace [2950,2212] has been stored successfully
AVO attributes for trace [2960,2192] has been stored successfully
AVO attributes for trace [2953,2223] has been stored successfully
AVO attributes for trace [2957,2182] has been stored successfully
AVO attributes for trace [2950,2213] has been stored successfully
AVO attributes for trace [2960,2193] has been stored successfully
AVO attributes for trace [2953,2224] has been stored successfully
AVO attributes for trace [2957,2183] has been stored successfully
AVO attributes for trace [2950,2214] has been stored successfully
AVO attrib

AVO attributes for trace [2954,2201] has been stored successfully
AVO attributes for trace [2951,2191] has been stored successfully
AVO attributes for trace [2957,2212] has been stored successfully
AVO attributes for trace [2960,2222] has been stored successfully
AVO attributes for trace [2954,2202] has been stored successfully
AVO attributes for trace [2951,2192] has been stored successfully
AVO attributes for trace [2957,2213] has been stored successfully
AVO attributes for trace [2960,2223] has been stored successfully
AVO attributes for trace [2954,2203] has been stored successfully
AVO attributes for trace [2951,2193] has been stored successfully
AVO attributes for trace [2957,2214] has been stored successfully
AVO attributes for trace [2960,2224] has been stored successfully
AVO attributes for trace [2954,2204] has been stored successfully
AVO attributes for trace [2951,2194] has been stored successfully
AVO attributes for trace [2957,2215] has been stored successfully
AVO attrib

AVO attributes for trace [2955,2181] has been stored successfully
AVO attributes for trace [2958,2192] has been stored successfully
AVO attributes for trace [2961,2202] has been stored successfully
AVO attributes for trace [2951,2223] has been stored successfully
AVO attributes for trace [2955,2182] has been stored successfully
AVO attributes for trace [2958,2193] has been stored successfully
AVO attributes for trace [2961,2203] has been stored successfully
AVO attributes for trace [2951,2224] has been stored successfully
AVO attributes for trace [2955,2183] has been stored successfully
AVO attributes for trace [2958,2194] has been stored successfully
AVO attributes for trace [2961,2204] has been stored successfully
AVO attributes for trace [2951,2225] has been stored successfully
AVO attributes for trace [2955,2184] has been stored successfully
AVO attributes for trace [2958,2195] has been stored successfully
AVO attributes for trace [2961,2205] has been stored successfully
AVO attrib

AVO attributes for trace [2962,2182] has been stored successfully
AVO attributes for trace [2958,2223] has been stored successfully
AVO attributes for trace [2952,2203] has been stored successfully
AVO attributes for trace [2955,2213] has been stored successfully
AVO attributes for trace [2962,2183] has been stored successfully
AVO attributes for trace [2958,2224] has been stored successfully
AVO attributes for trace [2952,2204] has been stored successfully
AVO attributes for trace [2955,2214] has been stored successfully
AVO attributes for trace [2962,2184] has been stored successfully
AVO attributes for trace [2958,2225] has been stored successfully
AVO attributes for trace [2952,2205] has been stored successfully
AVO attributes for trace [2955,2215] has been stored successfully
AVO attributes for trace [2962,2185] has been stored successfully
AVO attributes for trace [2958,2226] has been stored successfully
AVO attributes for trace [2952,2206] has been stored successfully
AVO attrib

...

AVO attributes for trace [2994,2229] has been stored successfully
AVO attributes for trace [2991,2219] has been stored successfully
AVO attributes for trace [2998,2187] has been stored successfully
AVO attributes for trace [2988,2210] has been stored successfully
AVO attributes for trace [2994,2230] has been stored successfully
AVO attributes for trace [2991,2220] has been stored successfully
AVO attributes for trace [2998,2188] has been stored successfully
AVO attributes for trace [2988,2211] has been stored successfully
AVO attributes for trace [2995,2180] has been stored successfully
AVO attributes for trace [2991,2221] has been stored successfully
AVO attributes for trace [2998,2189] has been stored successfully
AVO attributes for trace [2988,2212] has been stored successfully
AVO attributes for trace [2995,2181] has been stored successfully
AVO attributes for trace [2991,2222] has been stored successfully
AVO attributes for trace [2998,2190] has been stored successfully
AVO attrib

AVO attributes for trace [2995,2209] has been stored successfully
AVO attributes for trace [2998,2218] has been stored successfully
AVO attributes for trace [2989,2190] has been stored successfully
AVO attributes for trace [2995,2210] has been stored successfully
AVO attributes for trace [2992,2200] has been stored successfully
AVO attributes for trace [2998,2219] has been stored successfully
AVO attributes for trace [2989,2191] has been stored successfully
AVO attributes for trace [2992,2201] has been stored successfully
AVO attributes for trace [2995,2211] has been stored successfully
AVO attributes for trace [2998,2220] has been stored successfully
AVO attributes for trace [2989,2192] has been stored successfully
AVO attributes for trace [2992,2202] has been stored successfully
AVO attributes for trace [2995,2212] has been stored successfully
AVO attributes for trace [2998,2221] has been stored successfully
AVO attributes for trace [2989,2193] has been stored successfully
AVO attrib

AVO attributes for trace [2999,2198] has been stored successfully
AVO attributes for trace [2996,2190] has been stored successfully
AVO attributes for trace [2989,2221] has been stored successfully
AVO attributes for trace [2993,2180] has been stored successfully
AVO attributes for trace [2999,2199] has been stored successfully
AVO attributes for trace [2996,2191] has been stored successfully
AVO attributes for trace [2989,2222] has been stored successfully
AVO attributes for trace [2993,2181] has been stored successfully
AVO attributes for trace [2999,2200] has been stored successfully
AVO attributes for trace [2996,2192] has been stored successfully
AVO attributes for trace [2989,2223] has been stored successfully
AVO attributes for trace [2993,2182] has been stored successfully
AVO attributes for trace [2999,2201] has been stored successfully
AVO attributes for trace [2996,2193] has been stored successfully
AVO attributes for trace [2989,2224] has been stored successfully
AVO attrib

AVO attributes for trace [2996,2221] has been stored successfully
AVO attributes for trace [2990,2201] has been stored successfully
AVO attributes for trace [2993,2211] has been stored successfully
AVO attributes for trace [2999,2230] has been stored successfully
AVO attributes for trace [2996,2222] has been stored successfully
AVO attributes for trace [2990,2202] has been stored successfully
AVO attributes for trace [2993,2212] has been stored successfully
AVO attributes for trace [3000,2180] has been stored successfully
AVO attributes for trace [2996,2223] has been stored successfully
AVO attributes for trace [2990,2203] has been stored successfully
AVO attributes for trace [2993,2213] has been stored successfully
AVO attributes for trace [3000,2181] has been stored successfully
AVO attributes for trace [2996,2224] has been stored successfully
AVO attributes for trace [2990,2204] has been stored successfully
AVO attributes for trace [2993,2214] has been stored successfully
AVO attrib

AVO attributes for trace [2997,2201] has been stored successfully
AVO attributes for trace [2994,2191] has been stored successfully
AVO attributes for trace [3000,2210] has been stored successfully
AVO attributes for trace [2991,2182] has been stored successfully
AVO attributes for trace [2997,2202] has been stored successfully
AVO attributes for trace [2994,2192] has been stored successfully
AVO attributes for trace [3000,2211] has been stored successfully
AVO attributes for trace [2991,2183] has been stored successfully
AVO attributes for trace [2997,2203] has been stored successfully
AVO attributes for trace [2994,2193] has been stored successfully
AVO attributes for trace [3000,2212] has been stored successfully
AVO attributes for trace [2991,2184] has been stored successfully
AVO attributes for trace [2997,2204] has been stored successfully
AVO attributes for trace [2994,2194] has been stored successfully
AVO attributes for trace [3000,2213] has been stored successfully
AVO attrib

In [ ]:
with segyio.open(intercept_file, "r+") as intercept, segyio.open(gradient_file, "r+") as gradient:
    intercept

# Work with Segyio's iline

In [54]:
def ddd():
    with segyio.open(merged_stacks, "r") as f:
        x = f.iline[2950,36]
        return x

In [55]:
x = ddd()

In [56]:
x.shape[0]

51

In [58]:
with segyio.open(merged_stacks, "r") as f:
    print(f.iline[2950,12][0][750],f.iline[2950,24][0][750], f.iline[2950,36][0][750])

18.0 8.0 0.0


In [59]:
with segyio.open(merged_stacks, "r") as f:
    print(f.trace[0][750], f.trace[1][750], f.trace[2][750])

18.0 8.0 0.0


In [114]:
def xxx(iline_number, list_of_angles, cont):
    with segyio.open(merged_stacks, "r") as f:
        for cdp in range(0,2):#(f.iline[iline_number].shape[0]):
            #cunt = cont
            for time_slice in range(len(f.samples)):
                cdp_list = [] # contains the n np.array of amplitudes for one cdp
                for angle in list_of_angles:
                    cdp_list += [f.iline[iline_number, angle][cdp][time_slice]]
                
    return (cdp_list)

In [115]:
x = xxx(2950, angle_list, 0)

In [116]:
x

[0.0, 0.0, 0.0]

In [36]:
with segyio.open(merged_stacks, "r+") as f:
    print(f.iline[2950,12][0][750])
    f.iline[2950,12][0] = np.zeros([1501, ])
    
    print(f.iline[2950,12][0][750])
    f.iline[2950,12][1][750] = np.float32(0)
    
    print(f.iline[2950,12][1][750])
    f.trace[0] = np.zeros([1501, ])
    
    print(f.trace[0].max())
    #f.trace[0][750] = np.float32(1000)
    print(f.trace[0].max())

18.0
18.0
17.0
0.0
0.0


/home/lsd/anaconda3/envs/tesis/lib/python3.7/site-packages/segyio/utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


In [144]:
def fff():
    with segyio.open(merged_stacks, "r") as f:
        yield f.iline[2950]

In [145]:
x = fff()

In [146]:
cont = 0
for row in x: 
    print(f"HOLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA {cont}")
    print(row)
    cont +=1

HOLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA 0
[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.


# Third version of multiprocess avo attributes
The linear regression function might be giving a delay while computing the attributes because of how it works with slices instead of the entire array

## Making the SEG-Y files to store the AVO attributes [26 min]

In [9]:
def avo_storing_files(list_of_gathers, intercept_path, gradient_path): 
    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f:
        
        # Spec function to build the new segy
        spec = segyio.tools.metadata(f)

        # Initializing the merged one
        with segyio.create(gradient_path, spec) as gradient, segyio.create(intercept_path, spec) as intercept:

            # Assigning headers [byte]
            gradient.text[0], intercept.text[0] = f.text[0], f.text[0]
            gradient.bin, intercept.bin = f.bin, f.bin
            gradient.header, intercept.header = f.header, f.header
            gradient.trace, intercept.trace = f.trace, f.trace
            
    return (f"Successful construction. (Gradient, Intercept) path: ({intercept_path}, {gradient_path})")

In [10]:
avo_storing_files(gather_path, intercept_segy, gradient_segy)

'Successful construction. (Gradient, Intercept) path: (../data/intercept.sgy, ../data/gradient.sgy)'

## Function to give arguments to the pool

In [11]:
def input_generator(gather_file, list_of_angles, intercept_file, gradient_file):
    """
    
    """
    index = 0
    with segyio.open(gather_file, "r") as gather:
        for iline_number in gather.ilines:
            for xline_number in gather.xlines:
                    yield [gather_file, list_of_angles, iline_number, xline_number, index, intercept_file, gradient_file]
                    index += 1
                    
# test it
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(row)

## Function to compute the AVO attributes by trace

### Appending directly to a numpy array instead of making one trhough a list

In [ ]:
2 gb ram
4 nucleos (i3) // 2200

In [12]:
#gather_file = merged_stacks
#list_of_angles = angle_list
#iline_number = 2950
#xline_number = 2180

def computation_1(gather_file, list_of_angles, iline_number, xline_number):
    sins = []
    for angle in angle_list:
        sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]

    with segyio.open(gather_file, "r") as gather:
        grad = np.array([], dtype = "float32")
        inter = np.array([], dtype="float32")

        for time_slice in range(len(gather.samples)):
            amp = []
            for angle in list_of_angles:
                amp += [gather.gather[iline_number, xline_number, angle][time_slice]]

            # Regression
            gradient, intercept, r_value, p_value, std_error = stats.linregress(sins, amp)
            # Contribución: no están los parámetros enn un software pago
            # ===========================================================
            # Try with maxtaskperchild, chunks 
            #             
            
            grad = np.append(grad, gradient.astype("float32"))
            inter = np.append(inter, intercept.astype("float32"))

    return[inter, grad]

In [13]:
x = computation_1(merged_stacks, angle_list, 2950, 2180)[0]
x[0].dtype

dtype('float32')

### Using my own function of linear regression instead of scipy's

In [14]:
#gather_file = merged_stacks
#list_of_angles = angle_list
#iline_number = 2950
#xline_number = 2180
#amplitude_list = []

def computation_2(gather_file, list_of_angles, iline_number, xline_number):

    a_list = []
    for angles in list_of_angles:
        row = [1, np.sin(angles * np.pi/180)**2]
        a_list += [row]
    a = np.array(a_list)

    with segyio.open(gather_file, "r") as gather:
        grad = np.array([], dtype="float32" )
        inter = np.array([], dtype="float32" )
        for time_slice in range(len(gather.samples)):
            amp = []
            for angle in list_of_angles:
                amp += [gather.gather[iline_number, xline_number, angle][time_slice]]

            # Regression
            b = np.array(amp)
            # First term of avseth's equation
            first_term = np.dot(a.T, a)
            second_term = np.dot(a.T, b)
            # Third matrix: C ==> (At*A)**-1 * (At*b) = C
            c = np.dot(np.linalg.inv(first_term), second_term)

            grad = np.append(grad, c[1].astype("float32"))
            inter = np.append(inter, c[0].astype("float32"))

    return[inter, grad]

In [15]:
x = computation_2(merged_stacks, angle_list, 2950, 2180)[0]
x.shape

(1501,)

## Function to store the AVO attributes by trace

In [16]:
def store_AVO_attributes(generator_list):
    """
    
    """
    
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    
    to_store = computation_2(gather_file, list_of_angles, iline_number, xline_number)
    cont = 0
    with segyio.open(intercept_file, "r+") as intercept, segyio.open(gradient_file, "r+") as gradient:
       
        # Storing
        intercept.trace[index] = to_store[0]
        gradient.trace[index] = to_store[1]
        
        print(f"AVO attributes for trace [{iline_number},{xline_number}, {index}] has been stored successfully")
        return()
    
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(store_AVO_attributes(row))

In [17]:
def avo_storeputation(gather_file, list_of_angles, intercept_file, gradient_file):
    """
    https://stackoverflow.com/questions/30943161/multiprocessing-pool-with-maxtasksperchild-produces-equal-pids
    
    """
    input_data = input_generator(gather_file, list_of_angles, intercept_file, gradient_file)
    
    if __name__ == "__main__":
        p = Pool(4, maxtasksperchild = 1)
        p.map(store_AVO_attributes, input_data, chunksize = 1)

In [18]:
avo_storeputation(merged_stacks, angle_list, intercept_segy, gradient_segy)

AVO attributes for trace [2950,2181, 1] has been stored successfully
AVO attributes for trace [2950,2180, 0] has been stored successfully
AVO attributes for trace [2950,2182, 2] has been stored successfully
AVO attributes for trace [2950,2183, 3] has been stored successfully
AVO attributes for trace [2950,2186, 6] has been stored successfully
AVO attributes for trace [2950,2185, 5] has been stored successfully
AVO attributes for trace [2950,2184, 4] has been stored successfully
AVO attributes for trace [2950,2187, 7] has been stored successfully
AVO attributes for trace [2950,2189, 9] has been stored successfully
AVO attributes for trace [2950,2188, 8] has been stored successfully
AVO attributes for trace [2950,2190, 10] has been stored successfully
AVO attributes for trace [2950,2191, 11] has been stored successfully
AVO attributes for trace [2950,2192, 12] has been stored successfully
AVO attributes for trace [2950,2194, 14] has been stored successfully
AVO attributes for trace [2950

AVO attributes for trace [2952,2195, 117] has been stored successfully
AVO attributes for trace [2952,2196, 118] has been stored successfully
AVO attributes for trace [2952,2197, 119] has been stored successfully
AVO attributes for trace [2952,2198, 120] has been stored successfully
AVO attributes for trace [2952,2199, 121] has been stored successfully
AVO attributes for trace [2952,2200, 122] has been stored successfully
AVO attributes for trace [2952,2201, 123] has been stored successfully
AVO attributes for trace [2952,2202, 124] has been stored successfully
AVO attributes for trace [2952,2203, 125] has been stored successfully
AVO attributes for trace [2952,2204, 126] has been stored successfully
AVO attributes for trace [2952,2205, 127] has been stored successfully
AVO attributes for trace [2952,2206, 128] has been stored successfully
AVO attributes for trace [2952,2207, 129] has been stored successfully
AVO attributes for trace [2952,2208, 130] has been stored successfully
AVO at

AVO attributes for trace [2954,2211, 235] has been stored successfully
AVO attributes for trace [2954,2210, 234] has been stored successfully
AVO attributes for trace [2954,2209, 233] has been stored successfully
AVO attributes for trace [2954,2214, 238] has been stored successfully
AVO attributes for trace [2954,2212, 236] has been stored successfully
AVO attributes for trace [2954,2213, 237] has been stored successfully
AVO attributes for trace [2954,2215, 239] has been stored successfully
AVO attributes for trace [2954,2216, 240] has been stored successfully
AVO attributes for trace [2954,2217, 241] has been stored successfully
AVO attributes for trace [2954,2218, 242] has been stored successfully
AVO attributes for trace [2954,2219, 243] has been stored successfully
AVO attributes for trace [2954,2221, 245] has been stored successfully
AVO attributes for trace [2954,2220, 244] has been stored successfully
AVO attributes for trace [2954,2222, 246] has been stored successfully
AVO at

AVO attributes for trace [2956,2223, 349] has been stored successfully
AVO attributes for trace [2956,2224, 350] has been stored successfully
AVO attributes for trace [2956,2225, 351] has been stored successfully
AVO attributes for trace [2956,2226, 352] has been stored successfully
AVO attributes for trace [2956,2227, 353] has been stored successfully
AVO attributes for trace [2956,2228, 354] has been stored successfully
AVO attributes for trace [2956,2229, 355] has been stored successfully
AVO attributes for trace [2956,2230, 356] has been stored successfully
AVO attributes for trace [2957,2180, 357] has been stored successfully
AVO attributes for trace [2957,2181, 358] has been stored successfully
AVO attributes for trace [2957,2182, 359] has been stored successfully
AVO attributes for trace [2957,2183, 360] has been stored successfully
AVO attributes for trace [2957,2184, 361] has been stored successfully
AVO attributes for trace [2957,2185, 362] has been stored successfully
AVO at

AVO attributes for trace [2959,2186, 465] has been stored successfully
AVO attributes for trace [2959,2187, 466] has been stored successfully
AVO attributes for trace [2959,2188, 467] has been stored successfully
AVO attributes for trace [2959,2190, 469] has been stored successfully
AVO attributes for trace [2959,2189, 468] has been stored successfully
AVO attributes for trace [2959,2191, 470] has been stored successfully
AVO attributes for trace [2959,2192, 471] has been stored successfully
AVO attributes for trace [2959,2194, 473] has been stored successfully
AVO attributes for trace [2959,2193, 472] has been stored successfully
AVO attributes for trace [2959,2195, 474] has been stored successfully
AVO attributes for trace [2959,2196, 475] has been stored successfully
AVO attributes for trace [2959,2197, 476] has been stored successfully
AVO attributes for trace [2959,2198, 477] has been stored successfully
AVO attributes for trace [2959,2199, 478] has been stored successfully
AVO at

...

AVO attributes for trace [2990,2210, 2070] has been stored successfully
AVO attributes for trace [2990,2211, 2071] has been stored successfully
AVO attributes for trace [2990,2212, 2072] has been stored successfully
AVO attributes for trace [2990,2213, 2073] has been stored successfully
AVO attributes for trace [2990,2214, 2074] has been stored successfully
AVO attributes for trace [2990,2215, 2075] has been stored successfully
AVO attributes for trace [2990,2216, 2076] has been stored successfully
AVO attributes for trace [2990,2218, 2078] has been stored successfully
AVO attributes for trace [2990,2217, 2077] has been stored successfully
AVO attributes for trace [2990,2219, 2079] has been stored successfully
AVO attributes for trace [2990,2220, 2080] has been stored successfully
AVO attributes for trace [2990,2221, 2081] has been stored successfully
AVO attributes for trace [2990,2222, 2082] has been stored successfully
AVO attributes for trace [2990,2223, 2083] has been stored succe

AVO attributes for trace [2992,2221, 2183] has been stored successfully
AVO attributes for trace [2992,2223, 2185] has been stored successfully
AVO attributes for trace [2992,2224, 2186] has been stored successfully
AVO attributes for trace [2992,2225, 2187] has been stored successfully
AVO attributes for trace [2992,2226, 2188] has been stored successfully
AVO attributes for trace [2992,2227, 2189] has been stored successfully
AVO attributes for trace [2992,2228, 2190] has been stored successfully
AVO attributes for trace [2992,2229, 2191] has been stored successfully
AVO attributes for trace [2992,2230, 2192] has been stored successfully
AVO attributes for trace [2993,2180, 2193] has been stored successfully
AVO attributes for trace [2993,2181, 2194] has been stored successfully
AVO attributes for trace [2993,2183, 2196] has been stored successfully
AVO attributes for trace [2993,2182, 2195] has been stored successfully
AVO attributes for trace [2993,2184, 2197] has been stored succe

AVO attributes for trace [2995,2183, 2298] has been stored successfully
AVO attributes for trace [2995,2184, 2299] has been stored successfully
AVO attributes for trace [2995,2185, 2300] has been stored successfully
AVO attributes for trace [2995,2186, 2301] has been stored successfully
AVO attributes for trace [2995,2187, 2302] has been stored successfully
AVO attributes for trace [2995,2188, 2303] has been stored successfully
AVO attributes for trace [2995,2190, 2305] has been stored successfully
AVO attributes for trace [2995,2189, 2304] has been stored successfully
AVO attributes for trace [2995,2191, 2306] has been stored successfully
AVO attributes for trace [2995,2192, 2307] has been stored successfully
AVO attributes for trace [2995,2193, 2308] has been stored successfully
AVO attributes for trace [2995,2194, 2309] has been stored successfully
AVO attributes for trace [2995,2195, 2310] has been stored successfully
AVO attributes for trace [2995,2196, 2311] has been stored succe

AVO attributes for trace [2997,2195, 2412] has been stored successfully
AVO attributes for trace [2997,2197, 2414] has been stored successfully
AVO attributes for trace [2997,2198, 2415] has been stored successfully
AVO attributes for trace [2997,2196, 2413] has been stored successfully
AVO attributes for trace [2997,2199, 2416] has been stored successfully
AVO attributes for trace [2997,2200, 2417] has been stored successfully
AVO attributes for trace [2997,2201, 2418] has been stored successfully
AVO attributes for trace [2997,2202, 2419] has been stored successfully
AVO attributes for trace [2997,2203, 2420] has been stored successfully
AVO attributes for trace [2997,2205, 2422] has been stored successfully
AVO attributes for trace [2997,2204, 2421] has been stored successfully
AVO attributes for trace [2997,2206, 2423] has been stored successfully
AVO attributes for trace [2997,2207, 2424] has been stored successfully
AVO attributes for trace [2997,2209, 2426] has been stored succe

AVO attributes for trace [2999,2207, 2526] has been stored successfully
AVO attributes for trace [2999,2208, 2527] has been stored successfully
AVO attributes for trace [2999,2209, 2528] has been stored successfully
AVO attributes for trace [2999,2210, 2529] has been stored successfully
AVO attributes for trace [2999,2212, 2531] has been stored successfully
AVO attributes for trace [2999,2211, 2530] has been stored successfully
AVO attributes for trace [2999,2213, 2532] has been stored successfully
AVO attributes for trace [2999,2214, 2533] has been stored successfully
AVO attributes for trace [2999,2215, 2534] has been stored successfully
AVO attributes for trace [2999,2216, 2535] has been stored successfully
AVO attributes for trace [2999,2217, 2536] has been stored successfully
AVO attributes for trace [2999,2218, 2537] has been stored successfully
AVO attributes for trace [2999,2220, 2539] has been stored successfully
AVO attributes for trace [2999,2219, 2538] has been stored succe

In [21]:
3436 * 5052

17358672

In [34]:
# 100 traces ~~ per minute
# poseidon  3436 // 5052 ~~ 17.358.672 traces
to_finish_poseidon_sec = 3436 * 5052 * 60 /100 # == 10.415.203,2 seconds
to_finish_poseidon_min = to_finish_poseidon_sec /60 # 173586.72
to_finish_poseidon_hours = to_finish_poseidon_min / 60
to_finish_poseidon_days = to_finish_poseidon_hours/24
to_finish_poseidon_days

120.54633333333334

In [27]:
seconds_to_finish_poseidon_hours

2893.112

In [33]:
with segyio.open(near_stack, "r") as f:
    print(f.header[0])

{TRACE_SEQUENCE_LINE: 1, TRACE_SEQUENCE_FILE: 1, FieldRecord: 2950, TraceNumber: 0, EnergySourcePoint: 0, CDP: 2180, CDP_TRACE: 0, TraceIdentificationCode: 1, NSummedTraces: 0, NStackedTraces: 0, DataUse: 1, offset: 0, ReceiverGroupElevation: 0, SourceSurfaceElevation: 0, SourceDepth: 0, ReceiverDatumElevation: 0, SourceDatumElevation: 0, SourceWaterDepth: 0, GroupWaterDepth: 0, ElevationScalar: 0, SourceGroupScalar: -10, SourceX: 4237911, SourceY: 84893139, GroupX: 0, GroupY: 0, CoordinateUnits: 1, WeatheringVelocity: 0, SubWeatheringVelocity: 0, SourceUpholeTime: 0, GroupUpholeTime: 0, SourceStaticCorrection: 0, GroupStaticCorrection: 0, TotalStaticApplied: 0, LagTimeA: 0, LagTimeB: 0, DelayRecordingTime: 0, MuteTimeStart: 0, MuteTimeEND: 0, TRACE_SAMPLE_COUNT: 1501, TRACE_SAMPLE_INTERVAL: 4000, GainType: 0, InstrumentGainConstant: 0, InstrumentInitialGain: 0, Correlated: 0, SweepFrequencyStart: 0, SweepFrequencyEnd: 0, SweepLength: 0, SweepType: 0, SweepTraceTaperLengthStart: 0, Swe

# DelayRecordingTime: 0
- cube size
- chunks, maxtask
- store the other 3 scipy's parameters
- vectorize the for loops